In [1]:
import pandas as pd
import numpy as np

date = "1115"

tag_metadata_path = "../data/tag_metadata.xlsx"
orf_metadata_path = "../data/ORF_metadata.xlsx"

dataset_path = f"../data/{date}/{date}_all.xlsx"
ex_sheet = "ERM_REPA_1H"
tag_metadata = pd.read_excel(tag_metadata_path, index_col=0)
dataset_sheet = pd.read_excel(dataset_path, ex_sheet, usecols="A:D")
orf_metadata = pd.read_excel(orf_metadata_path)

In [7]:
def make_24karat(labels: pd.DataFrame, output_path:str) -> pd.DataFrame:
    df_24karat = pd.DataFrame(labels["Gene"].copy())
    df_24karat["24karat"] = pd.Series(dtype='int')
    df_24karat.loc[(labels["Classification_Orre2019"] == labels["Classification_ProteinAtlas"]) | (labels["Classification_Orre2019"] == labels["Classification_ProteinAtlas_grouped"]), "24karat"] = labels["Classification_Orre2019"]
    df_24karat.loc[df_24karat["24karat"] == 0, "24karat"]= np.nan
    df_24karat.set_index('Gene', inplace=True)
    return df_24karat

output_path = f"../data/{date}/{date}_str_labels.xlsx"
df_24karat = make_24karat(dataset_sheet, output_path)
df_24karat.to_excel(output_path)

In [8]:
def make_groundtruth(labels: pd.DataFrame, tag_metadata: pd.DataFrame, ignore_orfs: list, output_path:str) -> None:
    usable_tag_dict = tag_metadata["Label_Match"][tag_metadata["Label_Match"].notnull()].to_dict()
    gt_df = pd.DataFrame(index=labels.index.copy())

    for tag in usable_tag_dict:
        gt_df[f"ABE-{tag}"] = pd.Series(dtype='int')
        gt_df.loc[labels["24karat"] == usable_tag_dict[tag], f"ABE-{tag}"] = 1
        gt_df.loc[(labels["24karat"] != usable_tag_dict[tag]) & (labels["24karat"].notnull()), f"ABE-{tag}"] = 0
        gt_df.loc[gt_df.index.isin(ignore_orfs), f"ABE-{tag}"] = np.nan

    gt_df.to_excel(output_path)

inconsistent_mask = orf_metadata["Symbol"].duplicated()
inconsistent_orfs = set(orf_metadata[inconsistent_mask & orf_metadata["Valid_size?"]]["Symbol"]).intersection(set(orf_metadata[(inconsistent_mask) & (~orf_metadata["Valid_size?"])]["Symbol"]))
short_orfs = orf_metadata[~orf_metadata["Valid_size?"]]["Symbol"].to_list()
ignore_orfs = list(inconsistent_orfs) + short_orfs

output_path = f"../data/{date}/{date}_bin_labels.xlsx"
make_groundtruth(df_24karat, tag_metadata, ignore_orfs, output_path)

In [21]:
date = "1028"
output_path = f"../data/{date}/{date}_bin_labels.xlsx"
gt_df = pd.read_excel(output_path)
for col in gt_df.columns[1:]:
    print(f"---{col} Labels---")
    print (gt_df[col].value_counts(dropna=False).to_string(),"\n")

---ABE-NES Labels---
NaN    3250
0.0     508
1.0     169 

---ABE-OMM Labels---
NaN    3250
0.0     607
1.0      70 

---ABE-NLS Labels---
NaN    3250
0.0     426
1.0     251 

---ABE-MEM Labels---
NaN    3250
0.0     490
1.0     187 

